In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import cycle 
a = 'ì..ìì...................'
b = '...................ì..ìì'
print(a)
print(b)



def shift(s, d):
    if d == +1:
        if s[-1] == 'ì':
            return s
        else:
            shf = s[len(s)-1:] + s[:len(s)-1]
            return shf
    if d == -1:
        if s[0] == 'ì':
            return s
        else:
            shf = s[1:] + s[:1] 
            return shf
il = 0
ir = 0
dl = None
dr = None
bb1 = b
bb2 = b
while bb1 != a:
    il += 1
    nbb1 = shift(bb1, 1)
    print(nbb1)
    if nbb1 == a: 
        dl = il
        break
    if bb1 == nbb1: 
        print('wall')
        break
    else: bb1 = nbb1
     
while bb2 != a:
    ir += 1
    nbb2 = shift(bb2, -1)
    print(nbb2)
    if nbb2 == a:
        dr = ir 
        break
    if bb2 == nbb2: 
        print('wall')
        break
    else: bb2 = nbb2
    
print(dl, dr)

dist = dl if dl != None else dr 

print('register', dist)
print(a+'+'+b[::-1])

ì..ìì...................
...................ì..ìì
...................ì..ìì
wall
..................ì..ìì.
.................ì..ìì..
................ì..ìì...
...............ì..ìì....
..............ì..ìì.....
.............ì..ìì......
............ì..ìì.......
...........ì..ìì........
..........ì..ìì.........
.........ì..ìì..........
........ì..ìì...........
.......ì..ìì............
......ì..ìì.............
.....ì..ìì..............
....ì..ìì...............
...ì..ìì................
..ì..ìì.................
.ì..ìì..................
ì..ìì...................
None 19
register 19
ì..ìì...................+ìì..ì...................


In [3]:
import sys
sys.path.append('..')

import numpy as np

from hdna import *

In [4]:
stru = Structure('...(..(((.(....+......).)))..).')
print(stru.register)
stru.tails_l,stru.tails_r,stru.bulk

True ^ True

-2


False

In [5]:
struta = Structure(stru.table, fromtable=True)
print(struta.str == stru.str)
print(struta.left == stru.left)
print(struta.right == stru.right)


True
True
True


In [43]:
import networkx as nx 
M = Model(standard=True)
M.setgeometry(theta=90, phi=180)
A = Strand.random(M, 20)
# A = Strand(M, 'GACAAATTCGGCCTGGGA')
B = A.complementary()
K = Kinetics(M, A, B)
N = Kinetwork(M, A, B)    
print(A.sequence)
N.save_graph('.')

............((......+....)).............. --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
........((..........+.......))........... --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
....((..............+........)).......... --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
(((.................+..........)))....... --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
.((.................+.......))........... --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
...........((.......+.................)). --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
.......(((..........+.................))) --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
..........((........+..............)).... --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
...........((.......+..........))........ --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
..............((....+......))............ --> ((((((((((((((((((((+)))))))))))))))))))) pk duplex
............((......

In [8]:
N.filternodes()

TypeError: Kinetwork.filternodes() missing 3 required positional arguments: 'property', 'function', and 'graph'

In [ ]:
from itertools import permutations, combinations, pairwise

nodes = list(   N.filternodes('pairs', lambda x: x == 1, 
                N.filternodes('dpxdist', lambda x: x == -1, N.DG)).nodes())
nodes = [Structure(node) for node in nodes]
classifica = []
for packs in range(len(nodes)):
    for combo in combinations(nodes,packs):
        empty = Structure.empty(18)
        for node in combo:
            empty = [e1 or e2 for e1, e2 in zip(node.table, empty.table)]
            empty = Structure(empty, fromtable=True)
            compx = Complex(M, A, B.invert, state='backfray', structure=empty.str)
            classifica.append((empty.str, compx.G))

In [ ]:
import pandas as pd 

pd.DataFrame(classifica).drop_duplicates().sort_values(1)

,0,1
48,...............((.+..))..............,-2.546662
8,....((............+.............))...,-1.480401
153,............(..((.+..))..)...........,-1.051676
3,..........(.......+........).........,-0.488187
4,............(.....+......)...........,-0.488187
...,...,...
84,....(.....(.....(.+..).....).....)...,7.452172
265,.....(....(.(...(.+..)...).)....)....,7.768348
353,....(..(..(.(...(.+..)...).)..)..)...,8.067574
213,....(..(....(...(.+..)...)....)..)...,8.252785


In [ ]:
from itertools import combinations
edges = N.filteredges('state', lambda x: x in ['zipping', 'backfray'], N.DG)
fres = []
for e in edges.edges():
    fres.append(abs(N.DG.nodes[e[0]]['fre'] - N.DG.nodes[e[1]]['fre']))
np.mean(fres)

1.7395180139262645

In [ ]:
N.connect_slidings()

........((...+.)).......... 3 10 --> ..........((.+......))..... 10 6 pseudoknotting 7
..........((.+......))..... 10 6 --> ........((...+.)).......... 3 10 pseudoknotting back 7
........((...+.)).......... 3 10 --> .....((......+..........)). 5 10 pseudoknotting 7
.....((......+..........)). 5 10 --> ........((...+.)).......... 3 10 pseudoknotting back 7
........((...+.)).......... 3 10 --> ..........((.+...))........ 10 3 pseudoknotting 7
..........((.+...))........ 10 3 --> ........((...+.)).......... 3 10 pseudoknotting back 7
....(((......+...)))....... 6 7 --> .........((((+.........)))) 9 9 pseudoknotting 6
.........((((+.........)))) 9 9 --> ....(((......+...)))....... 6 7 pseudoknotting back 6
....(((......+...)))....... 6 7 --> ........((...+..........)). 8 10 pseudoknotting 6
........((...+..........)). 8 10 --> ....(((......+...)))....... 6 7 pseudoknotting back 6
....(((......+...)))....... 6 7 --> ..........((.+......))..... 10 6 pseudoknotting 6
..........((.+......))...

In [ ]:
nodz = list(N.filternodes('state', lambda x: x == 'on_nucleation', N.DG).nodes())

objs = [N.DG.nodes[node]['obj'] for node in nodz]

In [ ]:
for ob in objs:
    print(ob.structure)

(............+............)
.(...........+...........).
..(..........+..........)..
...(.........+.........)...
....(........+........)....
.....(.......+.......).....
......(......+......)......
.......(.....+.....).......
........(....+....)........
.........(...+...).........
..........(..+..)..........
...........(.+.)...........
............(+)............


In [ ]:
def get_ix(string, char):
    indices = []
    for i, e in enumerate(list(string)):
        if e == char:
            indices.append(i)
    return indices 

In [ ]:
on = 0
print(objs[on].structure)
l = objs[on].structure.split('+')[0]
r = objs[on].structure.split('+')[1][::-1]
ixl = get_ix(l,'(')[0]
ixr = get_ix(r,')')[0]

print(objs[on].s1.sdist[ixl])
objs[on].s1.sdist[ixl] == objs[on].s1.sdist[ixr]


(............+............)
0


True

In [ ]:
np.ceil(2*SXGEO.PERSISTENCE/SXGEO.MONODIST)

7.0

In [ ]:
print(f'{K.ksphere_sano():.3e}')

-6.350e+14


In [ ]:
bottomheignt = 2*SXGEO.PERSISTENCE
topgyradius  = K.gyradiuses(N=4)
kc = K.ksphere_sano(a=topgyradius, badd=(bottomheignt))
print(f'{kc:.3e}')
kc = K.ksphere_sano(SXGEO.CY_RADIUS)
print(f'{kc:.3e}')
print()
print(SXGEO.CY_RADIUS)
print(K.gr1)

-5.315e+14
-5.035e+14

1e-07
2.4373526622136567e-07
